In [4]:
# User parameters
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
color_dict = {'QB': 'red', 'RB': 'green', 'WR' : 'blue', 'TE' : 'orange'}

# Uncomment when needed
!pip install nfl_data_py==0.3.0
!pip install fuzzywuzzy==0.18.0

# Import packages
import nfl_data_py as nfl
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

# Define functions
def import_adp(years):
    adp_df = pd.DataFrame()
    for year in years:
        year_df = pd.read_csv(f'FantasyPros_{year}_Overall_ADP_Rankings.csv')
        year_df['adp_season'] = int(year)
        adp_df = pd.concat([adp_df, year_df], ignore_index=True)
    return adp_df

def fuzzy_merge(df_left, df_right, key_left, key_right, left_pos, right_pos, left_year, right_year, threshold=85):
    """
    This function merges two dataframes on specified columns first using hard matching and then fuzzy matching for those with no hard match.
    
    Args:
        df_left: the left table to join
        df_right: the right table to join
        key_left: key column of the left table
        key_right: key column of the right table
        threshold: how close the matches should be to return a match, based on Levenshtein distance
    
    Returns dataframe with boths keys and matches
    """
    # Convert key_right to list for fuzzy matching
    key_right_list = df_right[key_right].astype(str).tolist()

    # Hard merge first
    hard_merged_df = pd.merge(df_left, df_right, left_on=[key_left, left_pos, left_year], right_on=[key_right, right_pos, right_year], how='inner', suffixes=('', '_y'))

    # Filter the missing matches and drop the '_y' columns
    missing_matches = df_left[~df_left[key_left].isin(hard_merged_df[key_left])]
    fuzzy_df = missing_matches.copy()

    # Filter fuzzy_df to remove any duplicates in key_left that already exist in hard_merged_df
    fuzzy_df = fuzzy_df[~fuzzy_df[key_left].isin(hard_merged_df[key_left])]

    # Fuzzy match and merge
    fuzzy_matches = fuzzy_df[key_left].astype(str).apply(lambda x: process.extractOne(x, key_right_list, scorer=fuzz.token_sort_ratio, score_cutoff=threshold))
    fuzzy_df['fuzzy_matches'] = fuzzy_matches.apply(lambda x: x[0] if x else None)
    fuzzy_df['fuzzy_score'] = fuzzy_matches.apply(lambda x: x[1] if x else None)
    fuzzy_merged = pd.merge(fuzzy_df, df_right, left_on=['fuzzy_matches', left_pos, left_year], right_on=[key_right, right_pos, right_year], how='left', suffixes=('', '_y'))

    # Combine the results
    result = pd.concat([hard_merged_df, fuzzy_merged], axis=0)

    # Return dataframe
    return result

def calculate_bmi(weight, height):
    """
    Calculates the Body Mass Index (BMI) based on weight in pounds and height in inches.
    
    Args:
        weight (float): Weight of the person in pounds
        height (float): Height of the person in inches
        
    Returns the calculated BMI as float
    """
    bmi = (weight / (height ** 2)) * 703
    return bmi

# Imports from nfl_data_py
stats_df = nfl.import_seasonal_data(years)
roster_df = nfl.import_rosters(years)

# Imports from CSV
adp_df = import_adp(years)
forty_df = pd.read_csv('40_times.csv')

# Drop columns
#stats_df = stats_df.drop(['player_name', 'player_display_name', 'position', 'position_group', 'headshot_url'], axis=1)

# First merge and filter for fantasy positions only
final_df = pd.merge(roster_df, stats_df, on=['player_id', 'season'], how='right')
final_df = final_df[final_df['position'].isin(color_dict.keys())]

# Derive POSITION column for CSV
adp_df['POSITION'] = adp_df['POS'].apply(lambda x: str(x)[:2])

# Second merge for ADP using fuzzy matching
final_df = fuzzy_merge(final_df, adp_df, 'player_name', 'Player', 'position', 'POSITION', 'season', 'adp_season', threshold=85)

# Third merge for 40-times using fuzzy matching
final_df = fuzzy_merge(final_df, forty_df, 'player_name', 'player', 'position', 'position', 'entry_year', 'dash_year', threshold=85)

# Rename columns
final_df = final_df.rename(columns={'AVG': 'avg_adp'})

# Create columns
final_df['bmi'] = final_df.apply(lambda row: calculate_bmi(row['weight'], row['height']), axis=1)
final_df['player_season'] = final_df.apply(lambda row: f"{row['player_name']}, {row['season']:.0f}", axis=1)
final_df['draft_number_filled'] = final_df['draft_number'].fillna(225) # Fill missing draft_number with 225
final_df['draft_number_filled'] = pd.to_numeric(final_df['draft_number_filled'], errors='coerce')
final_df['draft_capital'] = final_df['draft_number_filled'].apply(lambda x: f"Round {((x-1) // 32) + 1}" if x <= 224 else "Undrafted")
final_df['bmi_over_30'] = final_df['bmi'].apply(lambda x: 1 if x > 30 else 0)
final_df['sub_4.6_40'] = final_df['40_yard_dash'].apply(lambda x: 1 if x < 4.6 else 0)

# Sort df by PPR points
final_df = final_df.sort_values('fantasy_points_ppr', ascending=False)
final_df = final_df.to_pickle('final_df.pkl')

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=37ec1120-bd1c-442f-b6e8-aab8eb5fc09e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>